In [1]:
import argparse
import ast
import itertools
import pathlib
import sys
import warnings

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

# plot the predictions
import seaborn as sns
import toml
from joblib import dump
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import ElasticNetCV, LogisticRegression, MultiTaskElasticNetCV

# import mse
from sklearn.metrics import mean_squared_error, r2_score

# import RepeatedKFold
from sklearn.model_selection import (
    GridSearchCV,
    LeaveOneOut,
    RepeatedKFold,
    StratifiedKFold,
    cross_val_score,
    train_test_split,
)
from sklearn.utils import parallel_backend

In [2]:
argparser = argparse.ArgumentParser()
argparser.add_argument("--cell_type", default="all")
argparser.add_argument("--shuffle", default="False")
argparser.add_argument("--cytokine", default="all")

args = argparser.parse_args()

cell_type = args.cell_type
shuffle = ast.literal_eval(args.shuffle)
cytokine = args.cytokine
print(cell_type, shuffle, cytokine)

In [4]:
# Parameters
aggregation = True
nomic = True

In [5]:
# set shuffle value
if shuffle == True:
    shuffle = "shuffled_baseline"
else:
    shuffle = "final"

In [6]:
MODEL_TYPE = "regression"

In [7]:
# load training data from indexes and features dataframe
data_split_path = pathlib.Path(
    f"../../0.split_data/indexes/{cell_type}/regression/aggregated_sc_and_nomic_data_split_indexes.tsv"
)
data_path = pathlib.Path(
    f"../../../data/{cell_type}_preprocessed_sc_norm_aggregated_nomic.parquet"
)

# dataframe with only the labeled data we want (exclude certain phenotypic classes)
data_df = pd.read_parquet(data_path)

data_split_indexes = pd.read_csv(data_split_path, sep="\t")

In [8]:
# rename column that contain the treatment dose to be a metadata column
data_df.rename(
    columns={
        "oneb_Treatment_Dose_Inhibitor_Dose": "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
    },
    inplace=True,
)

In [9]:
# remove duplicate columns
data_df = data_df.loc[:, ~data_df.columns.duplicated()]

In [10]:
# select tht indexes for the training and test set
train_indexes = data_split_indexes.loc[data_split_indexes["label"] == "train"]
test_indexes = data_split_indexes.loc[data_split_indexes["label"] == "test"]

In [11]:
# subset data_df by indexes in data_split_indexes
training_data = data_df.loc[train_indexes["labeled_data_index"]]
testing_data = data_df.loc[test_indexes["labeled_data_index"]]

In [12]:
# define metadata columns
# subset each column that contains metadata
metadata_train = training_data.filter(regex="Metadata")
# drop all metadata columns
train_data_x = training_data.drop(metadata_train.columns, axis=1)
train_treatments = training_data["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
# get all columns that contain "NSU" in the column name where NSU = normalized signal units
train_data_y_cols = train_data_x.filter(regex="NSU").columns
train_data_y = training_data[train_data_y_cols]
train_data_x = train_data_x.drop(train_data_y_cols, axis=1)


# define metadata columns
# subset each column that contains metadata
metadata_test = testing_data.filter(regex="Metadata")
# drop all metadata columns
test_data_x = testing_data.drop(metadata_test.columns, axis=1)
test_treatments = testing_data["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
# get all columns that contain "NSU" in the column name
test_data_y_cols = test_data_x.filter(regex="NSU").columns
test_data_y = testing_data[test_data_y_cols]
test_data_x = test_data_x.drop(test_data_y_cols, axis=1)

In [13]:
print(train_data_x.shape, train_data_y.shape, test_data_x.shape, test_data_y.shape)

(71, 1199) (71, 187) (83, 1199) (83, 187)


In [14]:
# set model path from parameters
if (aggregation == True) and (nomic == True):
    model_path = pathlib.Path(f"models/regression/{cell_type}/aggregated_with_nomic/")
elif (aggregation == True) and (nomic == False):
    model_path = pathlib.Path(f"models/regression/{cell_type}/aggregated/")
elif (aggregation == False) and (nomic == True):
    model_path = pathlib.Path(f"models/regression/{cell_type}/sc_with_nomic/")
elif (aggregation == False) and (nomic == False):
    model_path = pathlib.Path(f"models/regression/{cell_type}/sc/")
else:
    print("Error")

In [15]:
data_dict = {
    "train_data": {
        "data_x": train_data_x,
        "data_y": train_data_y,
        "col_names": train_data_y_cols,
        "metadata": metadata_train,
    },
    "test_data": {
        "data_x": test_data_x,
        "data_y": test_data_y,
        "col_names": test_data_y_cols,
        "metadata": metadata_test,
    },
}

In [16]:
# cross validation method
loo = LeaveOneOut()

# lsit of metrics to use
metrics = ["explained_variance", "neg_mean_absolute_error", "neg_mean_squared_error"]
output_metric_scores = {}

In [17]:
# blank df for concatenated results
results_df = pd.DataFrame(
    columns=[
        "explained_variance",
        "neg_mean_absolute_error",
        "neg_mean_squared_error",
        "well",
        "treatment",
        "r2",
        "cytokine",
        "data_split",
        "shuffle",
        "predicted_value",
        "actual_value",
        "log10_neg_mean_absolute_error",
        "log10_neg_mean_squared_error",
        "log10_explained_variance",
    ]
)

In [18]:
for data_split in data_dict:
    print(data_split)
    for i in data_dict[data_split]:
        print(i)
        data_x = data_dict[data_split]["data_x"]
        data_y = data_dict[data_split]["data_y"]
        col_names = data_dict[data_split]["col_names"]
        metadata = data_dict[data_split]["metadata"]
    if shuffle == "shuffled_baseline":
        model = joblib.load(
            f"../../1.train_models/{model_path}/{cytokine}_shuffled_baseline__all_nomic.joblib"
        )
    elif shuffle == "final":
        model = joblib.load(
            f"../../1.train_models/{model_path}/{cytokine}_final__all_nomic.joblib"
        )
    else:
        print("Error")

    # get the cytokine column of choice
    y_selected = data_y[cytokine]

    if shuffle == "shuffled_baseline":
        for column in data_x:
            np.random.shuffle(data_x[column].values)

    # get predictions
    predictions = model.predict(data_x)
    for metric in metrics:
        scores = cross_val_score(
            model, data_x, y_selected, scoring=metric, cv=loo, n_jobs=-1
        )
        output_metric_scores[metric] = scores
    r2 = r2_score(y_selected, predictions)
    output_metric_scores["treatment"] = metadata[
        "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
    ]
    output_metric_scores["well"] = metadata["Metadata_Well"].values
    df = pd.DataFrame.from_dict(output_metric_scores)
    df["r2"] = r2
    df["cytokine"] = cytokine
    df["data_split"] = data_split
    df["shuffle"] = shuffle
    df["predicted_value"] = predictions
    df["actual_value"] = y_selected
    df["log10_neg_mean_absolute_error"] = -np.log10(-df["neg_mean_absolute_error"])
    df["log10_neg_mean_squared_error"] = -np.log10(-df["neg_mean_squared_error"])
    df["log10_explained_variance"] = -np.log10(df["explained_variance"])

    # replace "[NSU]" with """
    df["cytokine"] = df["cytokine"].replace("[ \[\]NSU]", "", regex=True)
    df["cytokine"] = df["cytokine"].replace(" ", "_", regex=True)

    # concat the dataframes
    results_df = pd.concat([results_df, df], axis=0)

train_data
data_x
data_y
col_names
metadata


/tmp/ipykernel_411610/413048810.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, df], axis=0)


test_data
data_x
data_y
col_names
metadata


In [19]:
results_df.head()

,explained_variance,neg_mean_absolute_error,neg_mean_squared_error,well,treatment,r2,cytokine,data_split,shuffle,predicted_value,actual_value,log10_neg_mean_absolute_error,log10_neg_mean_squared_error,log10_explained_variance
5,1.0,-0.113421,-0.012864,B07,DMSO_0.100_%_DMSO_0.025_%,0.924271,ActivinA,train_data,final,0.153476,0.061116,0.945307,1.890614,-0.0
6,1.0,-0.030575,-0.000935,B08,LPS_0.010_ug_per_ml_DMSO_0.025_%,0.924271,ActivinA,train_data,final,0.769626,0.791261,1.514631,3.029261,-0.0
8,1.0,-0.256996,-0.066047,B10,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...,0.924271,ActivinA,train_data,final,0.204514,0.052033,0.590073,1.180147,-0.0
9,1.0,-0.057875,-0.003350,B11,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...,0.924271,ActivinA,train_data,final,0.104448,0.078860,1.237507,2.475014,-0.0
10,1.0,-0.194871,-0.037975,B12,Media,0.924271,ActivinA,train_data,final,0.199688,0.062455,0.710254,1.420508,-0.0


In [20]:
var_df = results_df.drop(
    columns=[
        "explained_variance",
        "neg_mean_absolute_error",
        "neg_mean_squared_error",
        "well",
        "treatment",
        "r2",
        "log10_neg_mean_absolute_error",
        "log10_neg_mean_squared_error",
        "log10_explained_variance",
    ]
)
# calculate the variance of the actual and predicted values per cytokine
var_df = var_df.groupby(["cytokine", "data_split", "shuffle"]).var()
var_df = pd.merge(
    var_df,
    results_df.groupby(["cytokine", "data_split", "shuffle"]).r2.unique(),
    left_index=True,
    right_index=True,
)
var_df.reset_index(inplace=True)
var_df.head()

,cytokine,data_split,shuffle,predicted_value,actual_value,r2
0,ActivinA,test_data,final,0.070876,0.088847,[0.8640594228782839]
1,ActivinA,train_data,final,0.047436,0.056173,[0.9242714043764793]


In [21]:
# set model path from parameters
if (aggregation == True) and (nomic == True):
    results_path = pathlib.Path(
        f"../results/regression/{cell_type}/aggregated_with_nomic/"
    )
elif (aggregation == True) and (nomic == False):
    results_path = pathlib.Path(f"../results/regression/{cell_type}/aggregated/")
elif (aggregation == False) and (nomic == True):
    results_path = pathlib.Path(f"../results/regression/{cell_type}/sc_with_nomic/")
elif (aggregation == False) and (nomic == False):
    results_path = pathlib.Path(f"../results/regression/{cell_type}/sc/")
else:
    print("Error")
pathlib.Path(results_path).mkdir(parents=True, exist_ok=True)

In [22]:
# check if the model training metrics file exists
metrics_file = pathlib.Path(f"{results_path}/{cytokine}_{shuffle}_model_stats.csv")

results_df.to_csv(metrics_file, index=False)

# do the same for the variance df
# check if the model training metrics file exists
metrics_file = pathlib.Path(
    f"{results_path}/{cytokine}_{shuffle}_variance_r2_stats.csv"
)
var_df.to_csv(metrics_file, index=False)